In [ ]:
#
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
start_of_train_year=2015
end_of_train_year=2019



In [ ]:
#demand
demand_folder=data_and_output_dir+'output/demand_aggreate' 
#each are store in os.path.join(output_folder_path, f'aggregated_demand_population_{year}.csv')
#column: 'reeds_ba'	'UTC' 'Time at End of Hour'	'Demand'	'Population'

#weather
weather_folder = data_and_output_dir +'output/'+'historical_weather'
# each file store in format of os.path.join(folder_to_store, f"{reeds_ba}_WRF_Hourly_Mean_Meteorology_{year}.csv")
#column:'Time_UTC' 'T2'	'Q2'	'SWDOWN'	'GLW'	'WSPD'

output_folder=data_and_output_dir +'output/'+'demand_weather_combine'
#Target file name:{reeds_ba}_WRF_Hourly_Mean_Meteorology.csv
#column:'Time_UTC' 'T2'	'Q2'	'SWDOWN'	'GLW'	'WSPD' 'Demand'	'Population'

In [ ]:
import pandas as pd
import os

# Loop through reeds_ba values
for reeds_ba_value in [f'p{i}' for i in range(1, 135)]:
    # Initialize a DataFrame to collect the yearly combined data
    combined_df_yearly = pd.DataFrame()

    # Find the demand and weather files for the current reeds_ba value
    for year in range(start_of_train_year, end_of_train_year+1):  # Adjust the year range as necessary
        demand_file = os.path.join(demand_folder, f'aggregated_demand_population_{year}.csv')
        weather_file = os.path.join(weather_folder, f"{reeds_ba_value}_WRF_Hourly_Mean_Meteorology_{year}.csv")

        # Check if both files exist for the current year
        if os.path.exists(demand_file) and os.path.exists(weather_file):
            demand_df = pd.read_csv(demand_file)
            weather_df = pd.read_csv(weather_file)

            # Filter demand data for the current reeds_ba
            demand_df_filtered = demand_df[demand_df['reeds_ba'] == reeds_ba_value]

            # Join demand and weather data on time columns
            combined_df = pd.merge(weather_df, demand_df_filtered, left_on='Time_UTC', right_on='UTC Time at End of Hour', how='inner')

            # Select and rename columns as required
            combined_df = combined_df[['Time_UTC', 'T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD', 'Demand', 'Population']]

            # Append to the yearly DataFrame
            combined_df_yearly = pd.concat([combined_df_yearly, combined_df], ignore_index=True)

    # After processing all years for the current reeds_ba, save the combined data
    if not combined_df_yearly.empty:
        output_file = os.path.join(output_folder, f"{reeds_ba_value}_historical_data.csv")
        # Convert 'Time_UTC' to datetime format
        combined_df_yearly['Time_UTC'] = pd.to_datetime(combined_df_yearly['Time_UTC'])

        # Extract year, month, day, and hour
        combined_df_yearly['Year'] = combined_df_yearly['Time_UTC'].dt.year
        combined_df_yearly['Month'] = combined_df_yearly['Time_UTC'].dt.month
        combined_df_yearly['Day'] = combined_df_yearly['Time_UTC'].dt.day
        combined_df_yearly['Hour'] = combined_df_yearly['Time_UTC'].dt.hour + 1  # Adjusting hour to 1-24 format

        # Rename columns
        combined_df_yearly.rename(columns={'Demand': 'Adjusted_Demand_MWh', 'Population': 'Total_Population'}, inplace=True)

        # Select and order the columns as required
        transformed_df = combined_df_yearly[['Year', 'Month', 'Day', 'Hour', 'Adjusted_Demand_MWh', 'Total_Population', 'T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']]

        combined_df_yearly.to_csv(output_file, index=False)
        print(f"Saved combined data for {reeds_ba_value}.")

print("All data processing complete.")
